# Model training

## Load data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D


In [2]:
outdir = "out_p_a1_given_q_xeff"
# import data
data = pd.read_pickle(f"{outdir}/p_a1_given_q_xeff.pkl")

In [3]:
data

,q,xeff,a1,p_a1
0,0.0,-1.0,"[0.0, 0.002004008016032064, 0.0040080160320641...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,0.0,-0.8,"[0.0, 0.002004008016032064, 0.0040080160320641...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,0.0,-0.6,"[0.0, 0.002004008016032064, 0.0040080160320641...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,0.0,-0.4,"[0.0, 0.002004008016032064, 0.0040080160320641...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,0.0,-0.2,"[0.0, 0.002004008016032064, 0.0040080160320641...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
...,...,...,...,...
116,1.0,0.2,"[0.0, 0.002004008016032064, 0.0040080160320641...","[1.0680033419862862, 1.0678605941877117, 1.067..."
117,1.0,0.4,"[0.0, 0.002004008016032064, 0.0040080160320641...","[0.7832345001188603, 0.7843442342552497, 0.784..."
118,1.0,0.6,"[0.2064128256513026, 0.2080031807101176, 0.209...","[0.0003581317510958827, 0.0006423405154926148,..."
119,1.0,0.8,"[0.6112224448897795, 0.6120015582266737, 0.612...","[0.003378646421929789, 0.004692188317429458, 0..."


### Some data contains nans, drop them so that they wont cause problems later

In [4]:
data.drop([0,1,2,3,4,5,6,7,8,9,10,11,21,22,32,33,43,44,54,55,65,66,76,77,87,88,98,99,109,110,120], inplace=True)

In [5]:
data.reset_index(drop=True, inplace=True)

## Spliting data into train and test

In [15]:
# split into train and test
from sklearn.model_selection import train_test_split
train_q, test_q, train_xeff, test_xeff, train_p_a1, test_p_a1, train_a1, test_a1 \
= train_test_split(data['q'], data['xeff'], data['p_a1'],data['a1'], test_size=0.2, random_state=42)

In [16]:
# convert to nparray
train_q = train_q.values
test_q = test_q.values
train_xeff = train_xeff.values
test_xeff = test_xeff.values
train_p_a1 = train_p_a1.values
test_p_a1 = test_p_a1.values
train_a1 = train_a1.values
test_a1 = test_a1.values

## Define model

In [14]:
# define model which take input of q and xeff and output p_a1 and a1
# input 2 dim, output 2 arrays
Input = keras.Input(shape=(2,))
dense = Dense(128, activation='relu')(Input)
dense = Dense(256, activation='relu')(dense)
output1 = Dense(500, activation='relu')(dense)
output2 = Dense(500, activation='relu')(dense)

model = Model(inputs=Input, outputs=[output1, output2])
model.compile(optimizer='adam',loss='mse')
history = model.fit([train_q, train_xeff], [train_p_a1, train_a1], epochs=100, batch_size=8)



ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).